# Лабораторная работа №1
## Задание:
1. Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).
2. Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Подключение библиотек


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных в датасет


In [3]:
Dataset = pd.read_csv('cybersecurity_attacks.csv')  # Замени на путь к файлу
print(Dataset.head())

             Timestamp Source IP Address Destination IP Address  Source Port  \
0  2023-05-30 06:33:58     103.216.15.12           84.9.164.252        31225   
1  2020-08-26 07:08:30    78.199.217.198         66.191.137.154        17245   
2  2022-11-13 08:23:25      63.79.210.48          198.219.82.17        16811   
3  2023-07-02 10:38:46     163.42.196.10        101.228.192.255        20018   
4  2023-07-16 13:11:07     71.166.185.76        189.243.174.238         6131   

   Destination Port Protocol  Packet Length Packet Type Traffic Type  \
0             17616     ICMP            503        Data         HTTP   
1             48166     ICMP           1174        Data         HTTP   
2             53600      UDP            306     Control         HTTP   
3             32534      UDP            385        Data         HTTP   
4             26646      TCP           1462        Data          DNS   

                                        Payload Data  ... Action Taken  \
0  Qui natus

### Создаём искомый столбец

In [4]:
label_encoder = LabelEncoder()
Dataset['Attack Type Encoded'] = label_encoder.fit_transform(Dataset['Attack Type'])
print(Dataset[['Attack Type', 'Attack Type Encoded']].head(10))

  Attack Type  Attack Type Encoded
0     Malware                    2
1     Malware                    2
2        DDoS                    0
3     Malware                    2
4        DDoS                    0
5     Malware                    2
6        DDoS                    0
7   Intrusion                    1
8   Intrusion                    1
9     Malware                    2


### Нормализация данных

In [5]:
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
columns_to_fill = ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores']
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404 entries, 0 to 9403
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Source Port          9404 non-null   int64  
 1   Destination Port     9404 non-null   int64  
 2   Packet Length        9404 non-null   int64  
 3   Anomaly Scores       9404 non-null   float64
 4   Attack Type Encoded  9404 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 367.5 KB


### Поиск лучшей метрики и количества соседей

In [6]:
X = Dataset.drop(columns=['Attack Type Encoded'])
y = Dataset['Attack Type Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': range(3, 40),
              'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_metric = best_params['metric']
best_k = best_params['n_neighbors']

accuracies = grid_search.cv_results_['mean_test_score']
accuracies = np.array(accuracies).reshape(len(param_grid['n_neighbors']), len(param_grid['metric']))

print(f"Лучшие параметры: Количество соседей = {best_k}, Лучшая метрика = {best_metric}")


Лучшие параметры: Количество соседей = 5, Лучшая метрика = chebyshev


#### Повторное обучение на лучших метриках и количестве соседей

In [7]:
knn = KNeighborsClassifier(n_neighbors=best_k, metric=best_metric)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели с 37 соседями и метрикой euclidean: {accuracy}")
report = classification_report(y_test, y_pred, zero_division = 1)
print(report)

Точность модели с 37 соседями и метрикой euclidean: 0.3272195640616693
              precision    recall  f1-score   support

           0       0.34      0.45      0.38      1267
           1       0.33      0.31      0.32      1286
           2       0.31      0.22      0.26      1209

    accuracy                           0.33      3762
   macro avg       0.32      0.33      0.32      3762
weighted avg       0.32      0.33      0.32      3762

